In [1]:
using MPSGE

Build the simple TWOBYTWO model:

In [2]:
m = Model()

@parameter(m, endow, 1.0)

@sector(m, X)
@sector(m, Y)
@sector(m, U)

@commodity(m, PX)
@commodity(m, PY)
@commodity(m, PU)
@commodity(m, PL)
@commodity(m, PK)

@consumer(m, RA, benchmark = 150.)

@production(m, X, 0, 1, [Output(PX, 100)], [Input(PL, 50), Input(PK, 50)])
@production(m, Y, 0, 1, [Output(PY, 50)], [Input(PL, 20), Input(PK, 30)])
@production(m, U, 0, 1, [Output(PU, 150)], [Input(PX, 100), Input(PY, 50)])

@demand(m, RA, 1., [Demand(PU, 150)], [Endowment(PL, :(70 * $endow)), Endowment(PK, 80.)])

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)


Next try to solve it with an iteration limit of 0 to replicate the benchmark:

In [3]:
solve!(m, cumulative_iteration_limit=0)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris



Preprocessed size   : 19

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             0.0000e+00             0.0e+00 (f[    1])

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (f[    1])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.156000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Did not solve with error: nothing.


Now change the value of the parameter `endow` and solve the model again:

In [4]:
set_value(endow, 1.1)
solve!(m)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 19

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             7.0613e+00             0.0e+00 (f[   20])
    1     1     0    19 7.0207e-01  1.0e+00    7.1e-02 (f[   20])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 7.0207e-01           I 7.0e-03 5.0e-01 (f[   20])
    1     1     3     3 1.3158e-03  1.0e+00 SO 2.8e-03 8.6e-04 (f[   20])
    2     1     4     4 1.0194e-07  1.0e+00 SO 1.3e-04 5.9e-08 (f[   11])

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 1.019364e-07
Postsolved residua

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Did not solve with error: nothing.


Now fix the `PX` variable at its benchmark value:

In [5]:
set_fixed!(PX, true)
solve!(m)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 18

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             7.0613e+00             0.0e+00 (f[   20])
    1     1     0    18 4.2701e-01  1.0e+00    0.0e+00 (f[    2])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 4.2701e-01           I 0.0e+00 2.8e-01 (f[    2])
    1     1     3     3 5.9085e-04  1.0e+00 SO 0.0e+00 3.4e-04 (f[    2])
    2     1     4     4 8.8207e-10  1.0e+00 SO 0.0e+00 5.4e-10 (f[    5])

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 8.820693e-10
Postsolved residua

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Did not solve with error: nothing.


Display an algebraic version of the model:

In [6]:
jm = algebraic_version(m)

  Compensated Demand
    (50.0 * ((PL†X / 50.0) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0)))) * ((((((((PL * (1.0 + +(0.0))) / (1.0 * 1.0)) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0)))) * (((PK * (1.0 + +(0.0))) / (1.0 * 1.0)) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0))))) * (((PL†X / 50.0) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0)))) * ((PK†X / 50.0) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0)))))) * 1.0 * 1.0) / (PL * (1.0 + +(0.0)))) ^ 1.0) * ((PK†X / 50.0) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0))))) - PL†X              ┴  PL†X
    (50.0 * ((PL†X / 50.0) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0)))) * ((((((((PL * (1.0 + +(0.0))) / (1.0 * 1.0)) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0)))) * (((PK * (1.0 + +(0.0))) / (1.0 * 1.0)) ^ ((50.0 * 1.0 * 1.0) / ((50.0 * 1.0 * 1.0) + (50.0 * 1.0 * 1.0))))) * (((PL†X